In [1]:
from loaders import *

In [3]:
config = dict(
    DRAM_factor_M=2048,
    DRAM_factor_N=2048,
    DRAM_factor_K=2048,
    DRAM_permutation=['K', 'N', 'M'],
    
    ClusterArray_factor_M=1,
    ClusterArray_factor_N=1,
    ClusterArray_factor_K=1,
    ClusterArray_permutation=['K', 'N', 'M'],
    
    GLB_Cluster_factor_M=1,
    GLB_Cluster_factor_N=1,
    GLB_Cluster_factor_K=1,
    GLB_Cluster_permutation=['K', 'N', 'M'],

    PE_Cluster_factor_M=1,
    PE_Cluster_factor_N=1,
    PE_Cluster_factor_K=1,
    PE_Cluster_permutation=['K', 'N', 'M'],

    iact_spad_factor_M=1,
    iact_spad_factor_N=1,
    iact_spad_factor_K=1,
    iact_spad_permutation=['K', 'N', 'M'],

    weight_spad_factor_M=1,
    weight_spad_factor_N=1,
    weight_spad_factor_K=1,
    weight_spad_permutation=['K', 'N', 'M'],

    psum_factor_M=1,
    psum_factor_N=1,
    psum_factor_K=1,
    psum_permutation=['K', 'N', 'M'],

    reg_factor_M=1,
    reg_factor_N=1,
    reg_factor_K=1,
    reg_permutation=['K', 'N', 'M'],

    density_weights=1, 
    density_inputs=1
) 

In [9]:
'''
Modeling lower triangular matrices: 
|--|--|
|Q1|Q2|
|--|--|
|Q4|Q3|
|__|__|

Q2: all-zeroes -- input density = 0
Q4: input density = 1 (actual values)
Q1 & Q3: input density = 0.5
'''

q1_config, q2_config, q4_config = config.copy(), config.copy(), config.copy()
q1_config['density_inputs'] = 0.5
q2_config['density_inputs'] = 0
q4_config['density_inputs'] = 1.0

In [ ]:
q1_out = run_timeloop_model(
    q1_config,
    problem='designs/attention/quadrant_problem.yaml',
    mapping='designs/baseline/baseline_mapping.yaml',
    sparse_optimizations='designs/baseline/baseline_sparse_opt.yaml', 
)
q1_stats = open('./output_dir/timeloop-model.stats.txt', 'r').read()

[INFO] 2025-04-28 17:33:51,973 - pytimeloop.accelergy_interface - Running Accelergy with command: accelergy /home/workspace/final_project/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v


INFO:pytimeloop.accelergy_interface:Running Accelergy with command: accelergy /home/workspace/final_project/output_dir/parsed-processed-input.yaml -o ./output_dir/ -v
